### Clean data script
A very quick script to process the data for .  Note that the data are so large and the EPA servers are so inconsistent that even the `requests` module with streaming data fails.  So just download the data to your local machine to process.  The objective is to just get started with some real data for California.

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# Grab and unzip the latest verion of the EJSCREEN data for 2020 in the `/tmp` folder.

# Found here (479MB): 
# https://gaftp.epa.gov/ejscreen/2020/EJSCREEN_2020_StatePctile.gdb.zip

gdf = gpd.read_file("/tmp/EJSCREEN_2020_StatePctile.gdb")

In [31]:
# Filter and save the results for just California.  It is still quite large, given 
# the boundaries of the Census tracts (approx. 263MB)
cali = gdf[gdf.ST_ABBREV == "CA"]
cali_attr = cali[["ID", "D_PM25_2", "geometry"]].reset_index(drop=True)

In [32]:
# Reproject to WGS84
cali_repr = cali_attr.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
cali_repr.to_file("/tmp/cali.geojson", driver="GeoJSON") 

In [43]:
# Create a CSV with the centroids of the tracts for quick web mapping
# TODO: clean up so that you don't have to force a reproject twice
geoms = cali_repr.geometry.to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
centroids = geoms.to_crs("EPSG:4326").centroid
cali_repr["lon"] = [pt.x for pt in centroids]
cali_repr["lat"] = [pt.y for pt in centroids]
cali_centroids = cali_repr[["ID", "D_PM25_2", "lon", "lat"]]
cali_centroids.to_csv("../data/cali_centroids.csv")

/Users/danhammer/streamlitenv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  after removing the cwd from sys.path.
